In [4]:
import requests

url = "https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001?Authorization=rdec-key-123-45678-011121314&format=JSON"
response = requests.get(url,verify = False)
if response.ok:
    print("下載成功")
    json_str:str = response.text
else:
    print("下載失敗")

下載成功


/home/vscode/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendata.cwb.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
import json

jsonData = json.loads(json_str)
jsonData

In [9]:
import json

jsonData1 = response.json()
print(jsonData1["success"])
print(jsonData1["records"]["datasetDescription"])

true
三十六小時天氣預報


In [ ]:
import json

jsonRecords = jsonData1['records']['location']
for item in jsonRecords:
    print(item)